In [1]:
#verify std
import statistics
import math
position=0
step=0
fee=0
t=242
step_list=[]
profit=0
mem=0
max_potential=0

while t<3200:
    #import gspread
    #from oauth2client.service_account import ServiceAccountCredentials
    # use creds to create a client to interact with the Google Drive API
    #scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    #creds = ServiceAccountCredentials.from_json_keyfile_name('Spreadsheet-5f9f3b27a502.json', scope)
    #gc= gspread.authorize(creds)
    #sh = gc.open('A new spreadsheet')
    #worksheet = sh.get_worksheet(0)
    import xlrd
    workbook = xlrd.open_workbook('cleaned_data.xls')
    s = workbook.sheet_by_index(0)
    
    
    #H_i=callback.historical_Data[(t-2)][3]
    H_i=s.cell((t-1),2).value
    #print(H_i) #1359.3
    #L_i=callback.historical_Data[(t-2)][4]
    L_i=s.cell((t-1),3).value
    #print(L_i) #1357.8
    #C_i=callback.historical_Data[(t-2)][5]
    C_i=s.cell((t-2),4).value
    #print(C_i)
    #O_i=callback.historical_Data[(t-2)][2]
    O_i=s.cell((t-2),1).value
    
    #H_j=callback.historical_Data[(t-1)][3]
    H_j=s.cell((t-1),2).value
    #print(H_j)
    #L_j=callback.historical_Data[(t-1)][4]
    L_j=s.cell((t-1),3).value
    #print(L_j)
    #C_j=callback.historical_Data[(t-1)][5]
    C_j=s.cell((t-1),4).value
    #print(C_j)
    #O_j=callback.historical_Data[(t-1)][2]
    O_j=s.cell((t-1),1).value
    
    #H_k=callback.historical_Data[t][3]
    H_k=s.cell(t,2).value
    #print(H_k) #1359.4
    #L_k=callback.historical_Data[t][4]
    L_k=s.cell(t,3).value
    #print(L_k) #1357.6
    #C_k=callback.historical_Data[t][5]
    C_k=s.cell(t,4).value
    #print(C_k) #1358.2
    O_k=s.cell(t,1).value
    
    
    #print("t"+str(t))
    #v=worksheet.acell('K'+str(s)).value
    v=s.cell((t-1),10).value
    
    #print(v)
    bbHj=float(v)-1.5
    #print(bollingerband_H_k) #1363.14
    w=s.cell((t-1),12).value
    #print(w)
    bbLj=float(w)+1.5
    
    #print(bollingerband_L_k) #1349.19
    
    q=s.cell((t-2),18).value
    #print(q)
    macd_his_i=float(q)
    e=s.cell((t-1),18).value
    macd_his_j=float(e)
    #print(e)
                     
    bbw_i=float(s.cell((t-2),19).value)
    #print(bbw_i)
    bbw_j=float(s.cell((t-1),19).value)
    #print(bbw_j)
    
    #check step
    if H_j<bbHj and L_j>bbLj:
        step=1
    elif H_j>bbHj and L_j>bbLj:
        step=2
    elif H_j<bbHj and L_j<bbLj:
        step =3
    else:
        step =0
    #print(step)
    
    sd=(bbw_j)/4
    sumforma=0
    n=3   #60:-19 40:-10 20:78
    m=80
    l=3
    for x in range(t-n, t): #print from t-240 to t-1
        h_s=s.cell(x,2).value
        l_s=s.cell(x,3).value
        hl2=(h_s+l_s)/2
        sumforma=sumforma+hl2
    sma=sumforma/n
    
    lj=[]
    for x in range(t-m, t): #print from t-240 to t-1
        h_s=s.cell(x,2).value
        l_s=s.cell(x,3).value
        hl2=(h_s+l_s)/2
        lj.append(hl2)
    sdm_j=statistics.stdev(lj)
    
    li=[]
    for x in range(t-m-1, t-1): #print from t-240 to t-1
        h_s=s.cell(x,2).value
        l_s=s.cell(x,3).value
        hl2=(h_s+l_s)/2
        li.append(hl2)
    sdm_i=statistics.stdev(li)
    
    C_p=float(s.cell(t-l,4).value)
    
    T=0
    initial=position
    final=position
    diff=0
        
    if t>243:
        T=t-242
        if position>0:
            if (L_k-mem)<=(-1.25):
                final=final-1
                if (H_k+L_k)/2-mem<=0:
                    diff=diff-1
                    
                
            elif macd_his_i>macd_his_j:  #18/48
                final=final-1
                
                if (H_k+L_k)/2-mem<=0:
                    diff = diff-(mem-((H_k+L_k)/2))
                elif (H_k+L_k)/2-mem>0:
                    diff = diff+(((H_k+L_k)/2)-mem)
                
            elif (H_k+L_k)/2-mem>1.2: #3/48
                if (H_k+L_k)/2-mem>max_potential:
                    max_potential=(H_k+L_k)/2-mem
                elif (H_k+L_k)/2-mem<max_potential*0.7:
                    final=final-1                    
                    max_potential=0
                    if (H_k+L_k)/2-mem<=0:
                        diff = diff-(mem-((H_k+L_k)/2))
                    elif (H_k+L_k)/2-mem>0:
                        diff = diff+(((H_k+L_k)/2)-mem)                    
            elif L_j<sma+1.5 or C_p>C_k: #23.8/48 or C_p>C_k: 1.8/23.8 L_j<sma+1.5:22
                final=final-1                
                if (H_k+L_k)/2-mem<=0:
                    diff = diff-(mem-((H_k+L_k)/2))
                elif (H_k+L_k)/2-mem>0:
                    diff = diff+(((H_k+L_k)/2)-mem)                
        if position<0:
            if (H_k-mem)>1.25:
                final=final+1
                if H_k-mem<=0:
                    diff = diff-(1)                
            elif macd_his_i<macd_his_j: #18/48
                final=final+1               
                if (H_k+L_k)/2-mem<=0:
                    diff = diff+(mem-((H_k+L_k)/2))
                elif (H_k+L_k)/2-mem>0:
                    diff = diff-(((H_k+L_k)/2)-mem)               
            elif mem-(H_k+L_k)/2>1.2:  #3/48
                if mem-(H_k+L_k)/2>max_potential:
                    max_potential=mem-(H_k+L_k)/2
                elif mem-(H_k+L_k)/2<max_potential*0.7:
                    final=final+1
                    max_potential=0
                    if (H_k+L_k)/2-mem<=0:
                        diff = diff+(mem-((H_k+L_k)/2))
                    elif (H_k+L_k)/2-mem>0:
                        diff = diff-(((H_k+L_k)/2)-mem)                    
            elif H_j>sma-1.5 or C_p<C_k:  #23.8/48 
                final=final+1
                if (H_k+L_k)/2-mem<=0:
                    diff = diff+(mem-((H_k+L_k)/2))
                elif (H_k+L_k)/2-mem>0:
                    diff = diff-(((H_k+L_k)/2)-mem)                
        if position==0:
            if C_p>C_k:  #36/48
                final=final-1
                
            elif C_p<C_k: #36/48
                final=final+1
                
            elif (H_j+L_j)/2<sma: #redundant
                final=final-1
                
            elif (H_j+L_j)/2>sma: #redundant
                final=final+1
                
    if final!=initial:
        fee=fee+1
        mem=(H_k+L_k)/2
        position=final
        profit=profit+diff
        
    if position!=0:
        print("position:"+str(position))
    print("profit:"+str(profit))
    print("mem:"+str(mem))
    print("(H_k+L_k)/2:"+str((H_k+L_k)/2))
    step_list.append(step)
    #print(step_list)
    min=1500
    max=0
    
    
    t = t+1
    print("T:"+str(T))
    print("t:"+str(t))
    #if t%88==0:
        #min=1500
        #max=0
    if (H_k+L_k)/2>max:
        max=(H_k+L_k)/2
    if (H_k+L_k)/2<min:
        min=(H_k+L_k)/2
    #print("---------------------------------------------------------------------------------------------------------")
    #if t%13==0:
        #time.sleep(100)
    print("fee:"+str(fee))
print("profit:"+str(profit))         
print(fee)

profit:0
mem:0
(H_k+L_k)/2:1356.95
T:0
t:243
fee:0
profit:0
mem:0
(H_k+L_k)/2:1357.55
T:0
t:244
fee:0
position:-1
profit:0
mem:1357.5
(H_k+L_k)/2:1357.5
T:2
t:245
fee:1
profit:1.2000000000000455
mem:1356.3
(H_k+L_k)/2:1356.3
T:3
t:246
fee:2
position:-1
profit:1.2000000000000455
mem:1356.4
(H_k+L_k)/2:1356.4
T:4
t:247
fee:3
profit:1.3500000000001364
mem:1356.25
(H_k+L_k)/2:1356.25
T:5
t:248
fee:4
position:1
profit:1.3500000000001364
mem:1356.55
(H_k+L_k)/2:1356.55
T:6
t:249
fee:5
profit:1.550000000000182
mem:1356.75
(H_k+L_k)/2:1356.75
T:7
t:250
fee:6
position:1
profit:1.550000000000182
mem:1357.05
(H_k+L_k)/2:1357.05
T:8
t:251
fee:7
profit:0.5500000000001819
mem:1355.85
(H_k+L_k)/2:1355.85
T:9
t:252
fee:8
position:-1
profit:0.5500000000001819
mem:1354.9
(H_k+L_k)/2:1354.9
T:10
t:253
fee:9
position:-1
profit:0.5500000000001819
mem:1354.9
(H_k+L_k)/2:1353.25
T:11
t:254
fee:9
position:-1
profit:0.5500000000001819
mem:1354.9
(H_k+L_k)/2:1353.1
T:12
t:255
fee:9
position:-1
profit:0.55000000